# <center>OpenAI agent pattern: routing</center>

A starter guide for building an agent loop using the `openai-agents` library.

This pattern uses routing to choose which specialized agent to use for a specific sub-task. The specialized agent attempts to complete the sub-task and return a result.

In the following example, we'll build an agent which creates a portfolio of stocks and ETFs based on a user's investment strategy.
1.  **Router Agent:** Chooses which worker to use based on the user's investment strategy.
2.  **Research Agent:** Searches the web for information about stocks and ETFs that could support the user's investment strategy.
3.  **Question Answering Agent:** Answers questions about investing like Warren Buffett.

### Install Libraries

In [ ]:
# Install base libraries for OpenAI
!pip install -q openai openai-agents

# Install optional libraries for OpenInference/OpenTelemetry tracing
!pip install -q arize-phoenix-otel openinference-instrumentation-openai-agents openinference-instrumentation-openai

### Setup Keys

Add your OpenAI API key to the environment variable `OPENAI_API_KEY`.

Copy your Phoenix `API_KEY` from your settings page at [app.phoenix.arize.com](https://app.phoenix.arize.com).

In [ ]:
import os
from getpass import getpass

os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"
if not os.environ.get("PHOENIX_CLIENT_HEADERS"):
    os.environ["PHOENIX_CLIENT_HEADERS"] = "api_key=" + getpass("Enter your Phoenix API key: ")

OPENAI_API_KEY = globals().get("OPENAI_API_KEY") or getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

### Setup Tracing

In [ ]:
from phoenix.otel import register

tracer_provider = register(
    project_name="openai-agents",
    endpoint="https://app.phoenix.arize.com/v1/traces",
    auto_instrument=True,
)

## Creating the agents

In [ ]:
from pprint import pprint
from textwrap import dedent

from agents import Agent, Runner, TResponseInputItem, WebSearchTool
from agents.model_settings import ModelSettings

qa_agent = Agent(
    name="Investing Q&A Agent",
    instructions=dedent("""You are Warren Buffett. You are answering questions about investing."""),
    model="gpt-4.1",
)

research_agent = Agent(
    name="Financial Search Agent",
    instructions=dedent(
        """You are a research assistant specializing in financial topics. Given a stock ticker, use web search to retrieve up‑to‑date context and produce a short summary of at most 50 words. Focus on key numbers, events, or quotes that will be useful to a financial analyst."""
    ),
    model="gpt-4.1",
    tools=[WebSearchTool()],
    model_settings=ModelSettings(tool_choice="required", parallel_tool_calls=True),
)

orchestrator_agent = Agent(
    name="Routing Agent",
    instructions=dedent(
        """You are a senior financial analyst. Your task is to handoff to the appropriate agent or tool."""
    ),
    model="gpt-4.1",
    handoffs=[
        research_agent,
        qa_agent,
    ],
)

In [ ]:
input_items: list[TResponseInputItem] = []

while True:
    user_input = input("Enter your question: ")
    if user_input == "exit":
        break
    input_item = {"content": user_input, "role": "user"}
    input_items.append(input_item)
    orchestrator = await Runner.run(orchestrator_agent, input_items)
    orchestrator_output = orchestrator.final_output
    pprint(orchestrator.last_agent)
    pprint(orchestrator_output)
    input_items = orchestrator.to_input_list()